In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mm import *
from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.lines as mlines

In [2]:
def load_parameter(filepath, evaluate=False):
    with open(filepath, 'r') as file:
        data = json.load(file)

    if evaluate:
        for parameter_list in [data["train_parameters"], data["test_parameters"]]:
            for parameter_set in parameter_list:
                parameter_set["radar_polangle"] = [[eval(angle) for angle in pair] for pair in parameter_set["radar_polangle"]]
                parameter_set["radar_altitude"] = [eval(value) for value in parameter_set["radar_altitude"]]
                parameter_set["warhead_v"] = [eval(value) for value in parameter_set["warhead_v"]]
                parameter_set["warhead_r"] = [eval(value) for value in parameter_set["warhead_r"]]
                parameter_set["track_interval"] = [eval(value) for value in parameter_set["track_interval"]]

    return data

In [19]:
# Load parameters.
parameters = load_parameter('parameters.json', evaluate=True)
# parameters['train_parameters'][0]
print(parameters['test_parameters'][0])

{'radar_altitude': [0], 'radar_polangle': [[0.6981317007977318, 0.5235987755982988]], 'warhead_v': [0, 3000.0, 1000.0], 'warhead_r': [6771000.0, 2000000.0, 4000000.0], 'track_interval': [120, 240]}


In [20]:
def display_track(initial_parameters_list, combine = True):
    R = 6371e3
    if combine:
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')

        u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
        x = 6371e3 * np.cos(u) * np.sin(v)
        y = 6371e3 * np.sin(u) * np.sin(v)
        z = 6371e3 * np.cos(v)
        ax.plot_surface(x, y, z, color='w', alpha=0.3, linewidth=1)

                # 添加经线和纬线
        for i in range(0, 360, 10):
            ax.plot(
                6371e3 * np.cos(np.radians(i)) * np.sin(v[:, 0]),
                6371e3 * np.sin(np.radians(i)) * np.sin(v[:, 0]),
                6371e3 * np.cos(v[:, 0]), 
                color='k', linestyle='--', linewidth=0.5
            )
            
        for i in range(-80, 90, 10):
            ax.plot(
                6371e3 * np.cos(u[0, :]) * np.sin(np.radians(i + 90)),
                6371e3 * np.sin(u[0, :]) * np.sin(np.radians(i + 90)),
                6371e3 * np.cos(np.radians(i + 90)), 
                color='k', linestyle='--', linewidth=0.5
            )
        # # ax.plot_surface(x, y, z, rstride=5, cstride=5, color='blue', alpha=0.5, linewidth=0.3, edgecolors='w')
        # # ax.plot_wireframe(x, y, z, color='k', linewidth=0.1)
        # ax.plot_surface(x, y, z, color='b', alpha=0.3, linewidth=0.5)
        # for parameters in initial_parameters_list:
        #     a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
        #     orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)

            
        #     view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, 'Warhead Orbit', orbit_points = orbit_points)
        #     ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
        #     ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
        #     start_point = orbit_points[parameters['track_interval'][0]]
        #     end_point = orbit_points[parameters['track_interval'][1]]

        #     for idx,radar_pa in enumerate(parameters['radar_polangle']):
        #         r = R + parameters['radar_altitude'][idx]
        #         radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
        #                             r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
        #                             r * np.sin(radar_pa[1])]) 
        #         ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
        #         ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
                
        #         ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
                
        #         # vertices = [ start_point, end_point, radar]
        #         # poly3d = [vertices]

        #         # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
                
    
    else:
        for num,parameters in enumerate(initial_parameters_list):
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(111, projection='3d')
            a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
            orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
            view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, None, orbit_points = orbit_points)
            ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
            ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
            start_point = orbit_points[parameters['track_interval'][0]]
            end_point = orbit_points[parameters['track_interval'][1]]

            for idx,radar_pa in enumerate(parameters['radar_polangle']):
                r = R + parameters['radar_altitude'][idx]
                radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[1])]) 
                ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
                ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
                
                ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
                
                # vertices = [radar, start_point, end_point]
                # poly3d = [vertices]

                # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
                verts = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]].tolist()
                verts.append(radar.tolist())
                ax.add_collection3d(Poly3DCollection([verts], facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))

                ax.set_axis_off()
            
            fig.savefig(f'./{num}.png', format='png', dpi=500, bbox_inches='tight') # change it.
            plt.close(fig)


%matplotlib qt
display_track(parameters['train_parameters'], combine = True)

In [21]:
def view_orbit(ax, r0, v0, radar, elev, azim ,title, orbit_points=None):
    # plot scatter
    if radar is None:
        pass
    else:
        ax.scatter(r0[0], r0[1], r0[2], color='black', s = 5)
        ax.scatter(radar[0], radar[1], radar[2], color='red', s = 5)
        
    ax.quiver(r0[0], r0[1], r0[2], v0[0], v0[1], v0[2], color='k', length=100, arrow_length_ratio=0.3, linewidth=1)


    # orbit
    if orbit_points is None:
        pass
    else:    
        ax.plot(orbit_points[:, 0], orbit_points[:, 1], orbit_points[:, 2],  color='r', alpha=1, linewidth=0.8, zorder = 50)
    
    # ax.set_box_aspect([1,1,1])

In [22]:
%matplotlib qt
type = 'train'

parameters = load_parameter('parameters.json', evaluate=True)
input = parameters[f'{type}_parameters']

R = 6371e3
# 创建一个新的3D图形
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# 设置角度以获得最佳视图
ax.view_init(elev=0, azim=45)

# 创建球体数据
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = R * np.outer(np.cos(u), np.sin(v))
y = R * np.outer(np.sin(u), np.sin(v))
z = R * np.outer(np.ones(np.size(u)), np.cos(v))

# 绘制白色的球体
ax.plot_surface(x, y, z, color='white', edgecolor='black', linewidth=0.1, alpha=0.3)

ax.set_axis_off()

for num,parameters in enumerate(input):
    a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
    orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
    view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, None, orbit_points = orbit_points)
    ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
    ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
    start_point = orbit_points[parameters['track_interval'][0]]
    end_point = orbit_points[parameters['track_interval'][1]]

    ob = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]]
    ax.plot(ob[:, 0], ob[:, 1], ob[:, 2], color='k', alpha=1, zorder = 100)

    for idx,radar_pa in enumerate(parameters['radar_polangle']):
        r = 6371e3 + parameters['radar_altitude'][idx]
        radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
                            r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
                            r * np.sin(radar_pa[1])]) 
        
        # ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
        # ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
        
        ax.scatter(radar[0], radar[1], radar[2] , color = 'b', s=5)
        
        # vertices = [radar, start_point, end_point]
        # poly3d = [vertices]

        # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
        # verts = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]].tolist()
        # verts.append(radar.tolist())
        # ax.add_collection3d(Poly3DCollection([verts], facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))


if type == 'test':
    ax.scatter([],[],[] , color = 'b',  alpha = 0.6,marker='.', label = 'Radar')
    ax.scatter([],[],[] , color = 'k', marker='x', alpha = 0.6, label = 'Reentry point')
    ax.scatter([],[],[] , color = 'k', marker='.', alpha = 0.6, label = 'Rlease point')
    ax.plot([],[],[],alpha=1,color='r' ,linewidth=0.8,label = 'Target orbit')
    ax.plot([],[],[],color='k', alpha=1,label = 'Observation interval')



    fig.legend(fontsize = 13, loc='center', bbox_to_anchor=(0.65, 0.39))
else:
    pass

# 显示图形（此时只有图例，没有其他内容）
plt.show()
ax.view_init(elev = 0, azim = 30)
ax.set_aspect('equal')
# 显示图形
plt.show()
plt.savefig(f'./{type}_orbit.png', dpi = 500)


img = Image.open(f'./{type}_orbit.png')

width, height = img.size

# 定义裁剪区域的边界（这里假设裁剪中间部分的宽度和高度分别是原图的50%）
left = width * 0.3
upper = height * 0.25
right = width * 0.83
lower = height * 0.75

# 裁剪图像
cropped_img = img.crop((left, upper, right, lower))
cropped_img.save(f'./{type}_orbit.png')

In [8]:
parameters.keys()

dict_keys(['radar_altitude', 'radar_polangle', 'warhead_v', 'warhead_r', 'track_interval'])

In [12]:
parameters['radar_altitude']

[0, 15]

In [13]:
# input = parameters['train_parameters']
for num,parameters in enumerate(input):
    print(num,parameters)
    break

0 {'radar_altitude': [0, 15], 'radar_polangle': [[0.7853981633974483, 0], [0.5235987755982988, 0.2617993877991494]], 'warhead_v': [0, 7120.0, 1000.0], 'warhead_r': [6771000.0, 0, 0], 'track_interval': [60, 120]}


In [ ]:
parameters['train_parameters']

TypeError: string indices must be integers, not 'str'

In [11]:
# import PIL
# import matplotlib.pyplot as plt
# import numpy as np
# from mpl_toolkits.mplot3d import Axes3D

# def plot_orbit(ax, r0, v0, orbit_points=None):
#     ax.scatter(r0[0], r0[1], r0[2], color='black', s = 5, zorder = 5)
#     ax.quiver(r0[0], r0[1], r0[2], v0[0], v0[1], v0[2], color='k', length=200, arrow_length_ratio=0.5, linewidth=1)

#     if orbit_points is None:
#         pass
#     else:    
#         ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k', zorder = 200)
#         ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k', zorder = 200)
#         ax.plot(orbit_points[:, 0], orbit_points[:, 1], orbit_points[:, 2], color='k', alpha=1, zorder = 90, depthshade=False)

    

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# R=6371e3
# # load bluemarble with PIL
# bm = PIL.Image.open('earth_texture.jpg')
# # it's big, so I'll rescale it, convert to array, and divide by 256 to get RGB values that matplotlib accept 
# # bm = np.array(bm.resize([d/5 for d in bm.size]))/256.
# bm = np.array(bm.resize([500,250]))/256
# # # coordinates of the image - don't know if this is entirely accurate, but probably close
# lons = np.linspace(-180, 180, bm.shape[1]) * np.pi/180 
# lats = np.linspace(-90, 90, bm.shape[0])[::-1] * np.pi/180 

# x = R*np.outer(np.cos(lons), np.cos(lats)).T
# y = R*np.outer(np.sin(lons), np.cos(lats)).T
# z = R*np.outer(np.ones(np.size(lons)), np.sin(lats)).T

# ax.plot_surface(x, y, z, rstride=10, cstride=10, facecolors = bm, edgecolor='black', linewidth=0.1, alpha=0.5, zorder = 1)



# for num,parameters in enumerate(input):
#     a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
#     orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
#     plot_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], orbit_points = orbit_points)
    

#     # break



#     start_point = orbit_points[parameters['track_interval'][0]]
#     end_point = orbit_points[parameters['track_interval'][1]]

#     for idx,radar_pa in enumerate(parameters['radar_polangle']):
#         r = 6371e3 + parameters['radar_altitude'][idx]
#         radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
#                             r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
#                             r * np.sin(radar_pa[1])]) 
#         # ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
#         # ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
        
#         ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
        
#         vertices = [radar, start_point, end_point]
#         poly3d = [vertices]

#         # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
#         # verts = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]].tolist()
#         # verts.append(radar.tolist())
#         # ax.add_collection3d(Poly3DCollection([verts], facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))

# # repeat code from one of the examples linked to in the question, except for specifying facecolors:



# ax.view_init(elev = 0, azim = 30)

# ax.set_axis_off()

# ax.set_aspect('equal')

In [10]:
bm = PIL.Image.open('earth_texture.jpg')
bm.show()
bm = bm.resize([500,100])
bm.show()

In [ ]:
np.linalg.norm(orbit_points[0])

7252230.139757011

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

R=6371e3

bm = PIL.Image.open('earth_texture.jpg')
bm = np.array(bm.resize([500,250]))/256
# # coordinates of the image - don't know if this is entirely accurate, but probably close
lons = np.linspace(-180, 180, bm.shape[1]) * np.pi/180 
lats = np.linspace(-90, 90, bm.shape[0])[::-1] * np.pi/180 

x = R*np.outer(np.cos(lons), np.cos(lats)).T
y = R*np.outer(np.sin(lons), np.cos(lats)).T
z = R*np.outer(np.ones(np.size(lons)), np.sin(lats)).T

ax.plot_surface(x, y, z, rstride=10, cstride=10, facecolors = bm, edgecolor='black', linewidth=0.1, alpha = 1, zorder=3)

for num, parameters in enumerate(input):
    a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
    orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
    view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, None, orbit_points = orbit_points)
    ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
    ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
    start_point = orbit_points[parameters['track_interval'][0]]
    end_point = orbit_points[parameters['track_interval'][1]]

    for idx,radar_pa in enumerate(parameters['radar_polangle']):
        r = 6371e3 + parameters['radar_altitude'][idx]
        radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
                            r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
                            r * np.sin(radar_pa[1])]) 
        # ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
        # ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
        
        ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
        
        vertices = [radar, start_point, end_point]
        poly3d = [vertices]

        # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
        # verts = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]].tolist()
        # verts.append(radar.tolist())
        # ax.add_collection3d(Poly3DCollection([verts], facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))

        break
